## NLP on Scraped FDA GOV Website
##### After removing foreign characters repeated texts etc.

#### Credits:
(http://pydata.org/dc2016/schedule/presentation/11/). To view the video of the presentation on YouTube, see [here](https://www.youtube.com/watch?v=6zm9NC9uRkk)._

## The FDA GOV dataset

In [1]:
import os
import codecs

data_directory = os.path.join('d:', 'IBM_WEX_FDA_GOV')

businesses_filepath = os.path.join(data_directory,
                                   'Text_Result.txt')

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

﻿          What We Do FDA-TRACK: sAgency-wide Program Performance A performance management system that 



In [2]:
intermediate_directory = os.path.join('d:', 'fda_gov_web_ibm_intermediate')

desc_txt_filepath = os.path.join(intermediate_directory,
                                   'details_all.txt')

In [3]:
%%time

if 1 == 1:
    row_count = 0

    # create & open a new file in write mode
    with codecs.open(desc_txt_filepath, 'w', encoding='utf_8') as desc_txt_file:

        # open the existing review json file
        with codecs.open(businesses_filepath, encoding='utf_8') as businesses_file:

            for line_txt in businesses_file.readlines():
                desc_txt_file.write(line_txt.replace('\n', '\\n') + '\n')
                row_count += 1

    print("Text from {:,} lines written to the new txt file.".format(row_count))
    
else:
    
    with codecs.open(desc_txt_filepath, encoding='utf_8') as desc_txt_file:
        for row_count, line in enumerate(desc_txt_filepath):
            pass
        
    print("Text from {:,} lines in the txt file.".format(row_count + 1))

Text from 5,246 lines written to the new txt file.
Wall time: 34 ms


## spaCy &mdash; Industrial-Strength NLP in Python

In [4]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

Let's grab a sample review to play with.

In [8]:
with codecs.open(desc_txt_filepath, encoding='utf_8') as f:
    sample_txt = list(it.islice(f, 3915, 3921))[0:5]
    sample_txt = ''.join(sample_txt).replace('\\n', '\n')
    
print(sample_txt)

          Committees Print this page Home Advisory Committees About Advisory 

          Committees The FDA uses 50 committees and panels to obtain 

          independent expert advice on scientific, technical, and policy 

          matters. About Advisory Committees Navigate the Advisory Committees 

          Section About Advisory Committees How to become a member of an 




Hand the review text to spaCy, and be prepared to wait...

In [9]:
%%time
parsed_txt = nlp(sample_txt)

Wall time: 10.2 ms


In [10]:
print(parsed_txt)

          Committees Print this page Home Advisory Committees About Advisory 

          Committees The FDA uses 50 committees and panels to obtain 

          independent expert advice on scientific, technical, and policy 

          matters. About Advisory Committees Navigate the Advisory Committees 

          Section About Advisory Committees How to become a member of an 




Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [11]:
for num, sentence in enumerate(parsed_txt.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
          Committees Print this page Home Advisory Committees About Advisory 

          Committees

Sentence 2:
The FDA uses 50 committees and panels to obtain 

          independent expert advice on scientific, technical, and policy 

          matters.

Sentence 3:
About Advisory Committees Navigate the Advisory Committees 

          Section

Sentence 4:
About Advisory Committees How to become a member of an 





What about named entity detection?

In [12]:
for num, entity in enumerate(parsed_txt.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1:           Committees Print - ORG

Entity 2: Home Advisory Committees About Advisory - ORG

Entity 3: FDA - ORG

Entity 4: 50 - CARDINAL

Entity 5: About Advisory Committees - ORG



What about part of speech tagging?

In [13]:
token_text = [token.orth_ for token in parsed_txt]
token_pos = [token.pos_ for token in parsed_txt]

pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,,SPACE
1,Committees,NOUN
2,Print,VERB
3,this,DET
4,page,NOUN
5,Home,PROPN
6,Advisory,PROPN
7,Committees,PROPN
8,About,ADP
9,Advisory,PROPN


What about text normalization, like stemming/lemmatization and shape analysis?

In [14]:
token_lemma = [token.lemma_ for token in parsed_txt]
token_shape = [token.shape_ for token in parsed_txt]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,,,
1,Committees,committee,Xxxxx
2,Print,print,Xxxxx
3,this,this,xxxx
4,page,page,xxxx
5,Home,home,Xxxx
6,Advisory,advisory,Xxxxx
7,Committees,committees,Xxxxx
8,About,about,Xxxxx
9,Advisory,advisory,Xxxxx


What about token-level entity analysis?

In [15]:
token_entity_type = [token.ent_type_ for token in parsed_txt]
token_entity_iob = [token.ent_iob_ for token in parsed_txt]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,,ORG,B
1,Committees,ORG,I
2,Print,ORG,I
3,this,,O
4,page,,O
5,Home,ORG,B
6,Advisory,ORG,I
7,Committees,ORG,I
8,About,ORG,I
9,Advisory,ORG,I


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [16]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_txt]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: 'Yes' if x else ''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,,-11.173082,,,Yes,,
1,Committees,-16.535004,,,,,
2,Print,-13.545116,,,,,
3,this,-5.361816,Yes,,,,
4,page,-9.092671,,,,,
5,Home,-11.417955,,,,,
6,Advisory,-15.521417,,,,,
7,Committees,-16.535004,,,,,
8,About,-10.633126,Yes,,,,
9,Advisory,-15.521417,,,,,


If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), spaCy is ready to use out-of-the-box.

I think it will eventually become a core part of the Python data science ecosystem &mdash; it will do for natural language computing what other great libraries have done for numerical computing.

## Phrase Modeling

_Phrase modeling_ is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so _new york_ would become *new\_york*). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as _happy hour_) to also become phrases in the model.

We turn to the indispensible [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling &mdash; the [**Phrases**](https://radimrehurek.com/gensim/models/phrases.html) class in particular.

In [17]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews in the `review_txt_all.txt` we created before
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [18]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_txt in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_txt.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [19]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

Let's use the `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text. We'll write this data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [20]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(desc_txt_filepath):
            f.write(sentence + '\n')

Wall time: 2.89 s


If your data is organized like our `unigram_sentences_all` file now is &mdash; a large text file with one document/sentence per line &mdash; gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components. It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once. This allows you to scale your modeling pipeline up to potentially very large corpora.

In [21]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [23]:
for unigram_sentence in it.islice(unigram_sentences, 3915, 3921):
    print(' '.join(unigram_sentence))
    print('')

bookmark this page because -PRON- will be update it on a regular basis

what ' new fda approves cinqair to treat severe asthma change

course a new approach to opioid pain medication at fda fda permit

marketing of device that sense optimal time to check patientâs

eye pressure fda provide $ 2 million in new grant for natural

history study in rare disease 2016



Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`ice cream`", to be linked together to form a new, single token: "`ice_cream`".

In [24]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [25]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 339 ms


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [26]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [27]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = ' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 477 ms


In [28]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [29]:
for bigram_sentence in it.islice(bigram_sentences,3915, 3921):
    print(' '.join(bigram_sentence))
    print('')

bookmark this_page because -PRON-_will be update it on a regular basis

what_' new fda approves cinqair to treat severe asthma change

course a new approach to opioid pain medication at fda fda permit

marketing of device that sense optimal time to check patientâs

eye pressure fda provide $ 2 million in new grant for natural

history study in rare disease 2016



Looks like the phrase modeling worked! We now see two-word phrases, such as "`ice_cream`" and "`apple_pie`", linked together in the text as a single token. Next, we'll train a _second-order_ phrase model. We'll apply the second-order phrase model on top of the already-transformed data, so that incomplete word combinations like "`vanilla_ice cream`" will become fully joined to "`vanilla_ice_cream`". No disrespect intended to [Vanilla Ice](https://www.youtube.com/watch?v=rog8ou-ZepE), of course.

In [30]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [31]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 291 ms


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [32]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [33]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = ' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 449 ms


In [34]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [35]:
for trigram_sentence in it.islice(trigram_sentences, 3915, 3921):
    print(' '.join(trigram_sentence))
    print('')

bookmark this_page because -PRON-_will be update it on a regular basis

what_'_new fda approves cinqair to treat severe asthma change

course a new approach to opioid pain medication at fda fda permit

marketing of device that sense optimal time to check patientâs

eye pressure fda provide $ 2 million in new grant for natural

history study in rare disease 2016



Looks like the second-order phrase model was successful. We're now seeing three-word phrases, such as "`vanilla_ice_cream`" and "`cinnamon_ice_cream`".

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point. _Stopwords_ are very common words, like _a_, _the_, _and_, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text. Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [36]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [37]:
 %%time
# %debug

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_desc in nlp.pipe(line_review(desc_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_desc
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
    
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.stop_words.STOP_WORDS]
            
            
            # write the transformed review as a line in the new file
            trigram_review = ' '.join(trigram_review)
            f.write(trigram_review + '\n')

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 3.2 s


Let's preview the results. We'll grab one review from the file with the original, untransformed text, grab the same review from the file with the normalized and transformed text, and compare the two.

In [38]:
print('Original:' + '\n')

for desc in it.islice(line_review(desc_txt_filepath), 23, 27):
    print(desc)

print('----' + '\n')
print('Transformed:' + '\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for desc in it.islice(f, 23, 27):
        print(desc)

Original:

          Reports, Manuals, &amp; Forms A collection of various FDA reports, 


          guides, and forms. About This Website FDA web policies, 


          accessibility, and support for mobile devices. Innovation at FDA The 


          latest news on FDA efforts to support innovation. Plain Language 


----

Transformed:

report manual amp forms collection fda report

guide form about_this_website fda web policy

accessibility support mobile device innovation fda

late_news fda effort support innovation plain language



You can see that most of the grammatical structure has been scrubbed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc. However, much of the general semantic *meaning* is still present. Also, multi-word concepts such as "`friday_night`" and "`above_average`" have been joined into single tokens, as expected. The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [39]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import _pickle as pickle

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [40]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [41]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 247 ms


Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [42]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [43]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for reason in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(reason)

In [44]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 312 ms


With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 50 topics.

In [45]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [46]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=10,
                           id2word=trigram_dictionary,
                           workers=1)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Wall time: 37.2 s


Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [47]:
def explore_topic(topic_number, topn=20):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format(u'term', u'frequency') + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=20):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [51]:
explore_topic(topic_number=0)

term                 frequency

fda                  0.102
report               0.018
information          0.017
safety               0.015
if_you               0.013
national             0.013
2015                 0.012
need                 0.011
issue                0.011
food                 0.010
medical_device       0.009
recall               0.009
use                  0.008
health               0.008
product              0.008
drug                 0.008
complaint            0.008
microsoft            0.008
list                 0.008
program              0.007


The first topic has strong associations with words like *taco*, *salsa*, *chip*, *burrito*, and *margarita*, as well as a handful of more general words. You might call this the **Mexican food** topic!

In [52]:
topic_names = {
        0: 'UNK0',
        1: 'UNK1',
        2: 'UNK2',
        3: 'UNK3',
        4: 'UNK4',
        5: 'UNK5',
        6: 'UNK6',
        7: 'UNK7',
        8: 'UNK8',
        9: 'UNK9'             
}

In [53]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

In [54]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [55]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, "rb") as f:
    LDAvis_prepared = pickle.load(f)

Wall time: 18.9 s


`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [57]:
pyLDAvis.display(LDAvis_prepared)

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [58]:
def get_sample_desc(desc):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(desc_txt_filepath),
                          desc, desc+1))[0]

In [59]:
def lda_description(desc_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_desc = nlp(desc_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_desc = [token.lemma_ for token in parsed_desc
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_desc = bigram_model[unigram_desc]
    trigram_desc = trigram_model[bigram_desc]
    
    # remove any remaining stopwords
    trigram_desc = [term for term in trigram_desc
                      if not term in spacy.en.stop_words.STOP_WORDS]
    
    # create a bag-of-words representation
    desc_bow = trigram_dictionary.doc2bow(trigram_desc)
    
    # create an LDA representation
    desc_lda = lda[desc_bow]
    
    # sort with the most highly related topics first
    desc_lda = sorted(desc_lda, key=lambda topic_lda: -topic_lda[1])
    
    for topic_number, freq in desc_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:2} {}'.format(topic_names[topic_number],
                                round(freq, 2)))

In [62]:
sample_desc = get_sample_desc(89)
print(sample_desc)

          appropriate based on surrogate or intermediate endpoints reasonably 




In [63]:
lda_description(sample_desc)

UNK3 0.7


C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [67]:
sample_desc= get_sample_desc(313)
print(sample_desc)

          Nutrition Month Learn how the changes to the Nutrition Facts Label 




In [68]:
lda_description(sample_desc)

UNK9 0.5
UNK1 0.39


C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## Word Vector Embedding with Word2Vec

Pop quiz! Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [69]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [70]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    desc2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=1)
    
    desc2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        desc2vec.train(trigram_sentences)
        desc2vec.save(word2vec_filepath)
        
# load the finished model from disk
desc2vec = Word2Vec.load(word2vec_filepath)
desc2vec.init_sims()

print('{} training epochs so far.'.format(desc2vec.train_count))

12 training epochs so far.
Wall time: 5.86 s


In [71]:
print('{:,} terms in the recall2vec vocabulary.'.format(len(desc2vec.wv.vocab)))

342 terms in the recall2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [72]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in desc2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda term_index: -term_index[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(desc2vec.wv.syn0[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
and,-0.100746,-0.138351,-0.065297,0.005219,-0.064796,0.121452,0.006101,0.318425,-0.126467,-0.380278,...,0.096713,-0.085560,0.658379,-0.261223,-0.071542,-0.181564,-0.014139,-0.078411,-0.020709,-0.003840
the,-0.018843,-0.254019,0.248716,-0.190241,-0.161016,0.074082,-0.170656,-0.169396,0.166634,-0.168215,...,-0.044964,-0.344766,0.553132,-0.360908,-0.294987,0.203086,-0.141143,-0.366750,-0.245072,-0.123179
fda,0.077440,-0.207896,0.269897,0.162306,-0.050462,0.209877,-0.053269,0.277357,-0.344069,-0.206898,...,-0.087527,-0.177071,0.737522,-0.076948,-0.137575,-0.101029,-0.451010,0.197468,-0.017319,0.105784
to,-0.313661,-0.045335,0.089928,-0.039525,-0.035883,0.346876,0.005409,0.336155,-0.142290,-0.306906,...,0.022846,-0.148466,0.072798,-0.228918,0.020430,-0.064480,0.007564,-0.602445,-0.077429,-0.071003
of,-0.001229,0.145161,-0.044128,0.021164,-0.402606,-0.152231,-0.153400,0.136081,0.073617,-0.055965,...,-0.155064,-0.245643,0.417786,-0.173233,0.190563,-0.218247,-0.337335,-0.130735,0.017617,-0.065531
for,0.042443,0.284259,-0.266208,0.018548,-0.168083,0.183761,-0.005107,0.149055,0.020657,-0.257668,...,-0.319383,0.010611,0.213731,0.004790,-0.263443,-0.361939,-0.567995,-0.588227,-0.011243,0.094405
be,0.123699,-0.042003,-0.086108,-0.259822,-0.393856,0.329685,-0.109405,-0.234873,-0.156663,-0.137926,...,-0.382407,-0.409366,0.301657,-0.129887,-0.080969,0.249293,0.010820,-0.530850,-0.094270,-0.052313
a,-0.141158,-0.072698,0.483275,-0.025337,-0.195636,0.584957,-0.140417,0.080549,-0.007860,-0.360949,...,0.057271,-0.366956,0.507431,-0.140168,-0.306811,0.294925,-0.060049,-0.188385,-0.162945,-0.171217
in,0.117567,-0.305639,0.376731,-0.012498,-0.344085,0.148888,0.235434,0.266651,0.153096,-0.142239,...,-0.518270,0.122086,0.506006,-0.362484,-0.330909,-0.322330,-0.178481,-0.121798,0.535190,-0.152664
information,0.205563,0.389818,-0.072361,0.305935,-0.253689,0.419405,-0.259644,-0.377517,0.043917,-0.027382,...,0.128690,-0.172287,0.063882,0.148540,-0.184837,-0.053577,0.385220,-0.080125,-0.290900,-0.150123


### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [73]:
def get_related_terms(token, topn=5):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in desc2vec.most_similar(positive=[token], topn=topn):

        print('{:20} {}'.format(word, round(similarity, 3)))

###

In [74]:
get_related_terms('health')

international_programs 0.442
globalization        0.435
animal               0.396
u.s.                 0.388
cosmetics            0.382


### 

In [75]:
get_related_terms('tobacco')

cigarette            0.497
control              0.438
radiation            0.423
keep                 0.406
video                0.404


In [76]:
get_related_terms('blood')

cell                 0.609
tissue               0.595
therapy              0.486
certain              0.472
biologics            0.461


### 

In [77]:
get_related_terms('radiation', topn=20)

control              0.532
medwatch             0.491
radiation_emitting_products 0.485
reports              0.476
inspection           0.468
electronic           0.443
assistance           0.433
submit               0.429
tobacco              0.423
icsrs                0.412
online               0.41
updates              0.408
response             0.406
regulatory_information 0.399
datum                0.394
international_programs 0.394
manufacturer         0.389
state                0.383
innovation           0.376
medical              0.375


In [78]:
get_related_terms('disease', topn=20)

treat                0.534
history              0.473
colorectal_cancer    0.467
treatment            0.437
clinical_trial       0.428
first                0.412
approve              0.411
certain              0.405
april                0.399
u.s.                 0.393
late                 0.39
risk                 0.389
february             0.385
before               0.384
m.d.                 0.38
national             0.376
opportunity          0.376
more_than            0.374
commissioner         0.374
test                 0.371


## Word algebra!
No self-respecting word2vec demo would be complete without a healthy dose of *word algebra*, also known as *analogy completion*.

The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

In [79]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = desc2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

### 

In [80]:
word_algebra(add=['devices', 'radiation'])

reports


### 

In [81]:
word_algebra(add=['new', 'drug'], subtract=['animal'])

clinical


Now we're getting a bit more nuanced. The model has discovered that:
- Both *lunch* and *dinner* are meals
- The main difference between them is time of day
- Day and night are times of day
- Lunch is associated with day, and dinner is associated with night

What else?

### 

In [82]:
word_algebra(add=['product', 'tobacco'], subtract=['drug'])

cigarette


## Word Vector Visualization with t-SNE

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [83]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [84]:
tsne_input = word_vectors.drop(spacy.en.stop_words.STOP_WORDS, errors='ignore')
tsne_input = tsne_input.head(1000)

In [85]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
fda,0.077440,-0.207896,0.269897,0.162306,-0.050462,0.209877,-0.053269,0.277357,-0.344069,-0.206898,...,-0.087527,-0.177071,0.737522,-0.076948,-0.137575,-0.101029,-0.451010,0.197468,-0.017319,0.105784
information,0.205563,0.389818,-0.072361,0.305935,-0.253689,0.419405,-0.259644,-0.377517,0.043917,-0.027382,...,0.128690,-0.172287,0.063882,0.148540,-0.184837,-0.053577,0.385220,-0.080125,-0.290900,-0.150123
drug,-0.360087,0.276967,-0.217438,-0.554882,0.135724,0.015508,-0.177810,0.301658,0.196800,0.180682,...,-0.241992,-0.422613,-0.096184,-0.014281,-0.358708,0.127414,0.215810,-0.090970,-0.330258,-0.244218
food,0.397796,-0.316026,-0.421999,-0.214100,-0.304859,-0.145224,-0.686528,0.363681,0.023887,0.143341,...,0.473187,-0.202776,0.423708,0.389340,-0.423926,-0.001822,-0.120035,-0.066002,0.069482,0.010907
safety,0.487511,-0.051075,-0.222669,-0.172921,0.134228,-0.374469,0.048707,0.131852,-0.418310,-0.020593,...,0.291371,0.527957,0.115536,-0.268350,-0.239712,-0.113702,-0.277039,-0.497297,-0.129665,0.271672


In [86]:
tsne_filepath = os.path.join(intermediate_directory,
                             'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     'tsne_vectors.npy')

In [87]:
%%time

if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, "rb") as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=['x_coord', 'y_coord'])

Wall time: 2.99 s


Now we have a two-dimensional representation of our data! Let's take a look.

In [88]:
tsne_vectors.head()

,x_coord,y_coord
fda,36.795262,-2.148801
information,-0.608412,12.562070
drug,20.612260,-19.267174
food,6.174195,-2.198315
safety,11.311978,-6.762396


In [89]:
tsne_vectors['word'] = tsne_vectors.index

### Plotting with Bokeh

In [90]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\bokeh\core\json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Loading BokehJS ...

In [91]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom,'
                           'box_select, resize, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering